<h1>Thu thấp các nước đi<h1>

In [ ]:
import tkinter as tk
import pandas as pd

class CaroGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Caro Game - Thu Thập Dữ Liệu")

        self.board_size = None
        self.win_condition = None
        self.moves = []
        self.current_player = 1
        self.board = None
        self.game_over = False

        # Menu chọn kích thước bàn cờ
        self.menu_frame = tk.Frame(root)
        self.menu_frame.pack()
 
        tk.Label(self.menu_frame, text="Chọn kích thước bàn cờ:").pack()
        tk.Button(self.menu_frame, text="20x20 (5 quân thắng)", command=lambda: self.start_game(20, 5)).pack()
        tk.Button(self.menu_frame, text="5x5 (3 quân thắng)", command=lambda: self.start_game(5, 3)).pack()
        tk.Button(self.menu_frame, text="Thoát", command=self.exit_game).pack()  # Thêm nút thoát

    def start_game(self, size, win_condition):
        """Khởi động game với kích thước và điều kiện thắng tương ứng"""
        self.board_size = size
        self.win_condition = win_condition
        self.moves = []
        self.current_player = 1
        self.board = [[0] * size for _ in range(size)]
        self.game_over = False

        # Xóa menu chọn kích thước
        for widget in self.menu_frame.winfo_children():
            widget.destroy()

        # Tạo lại menu sau khi xóa các widget cũ
        self.canvas = tk.Canvas(self.root, width=size * 40, height=size * 40, bg="white")
        self.canvas.pack()
        self.canvas.bind("<Button-1>", self.on_click)

        self.draw_board()

        # Nút lưu dữ liệu
        self.save_button = tk.Button(self.root, text="Lưu dữ liệu", command=self.save_moves)
        self.save_button.pack()

    def draw_board(self):
        """Vẽ bàn cờ"""
        for i in range(self.board_size):
            for j in range(self.board_size):
                x1, y1 = j * 40, i * 40
                x2, y2 = x1 + 40, y1 + 40
                self.canvas.create_rectangle(x1, y1, x2, y2, outline="black")

    def on_click(self, event):
        """Xử lý khi người chơi click vào bàn cờ"""
        if self.game_over:
            return  # Nếu đã có kết quả, không cho phép tiếp tục

        row, col = event.y // 40, event.x // 40
        if self.board[row][col] == 0:  # Kiểm tra ô trống
            self.board[row][col] = self.current_player
            self.moves.append([len(self.moves) + 1, self.current_player, row, col])

            # Vẽ quân cờ
            color = "black" if self.current_player == 1 else "red"
            self.canvas.create_oval(col * 40 + 5, row * 40 + 5, col * 40 + 35, row * 40 + 35, fill=color)

            # Kiểm tra thắng
            if self.check_win(row, col):
                self.save_moves(winner=self.current_player)
                self.show_winner(self.current_player)
                self.game_over = True
                return

            # Kiểm tra hòa (hết bàn cờ)
            if len(self.moves) == self.board_size * self.board_size:
                self.save_moves(winner=0)
                self.show_draw()
                self.game_over = True
                return

            # Chuyển lượt chơi
            self.current_player = 3 - self.current_player  

    def check_win(self, row, col):
        """Kiểm tra xem người chơi hiện tại có thắng không"""
        directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
        for dx, dy in directions:
            count = 1
            for i in range(1, self.win_condition):
                r, c = row + i * dx, col + i * dy
                if 0 <= r < self.board_size and 0 <= c < self.board_size and self.board[r][c] == self.current_player:
                    count += 1
                else:
                    break

            for i in range(1, self.win_condition):
                r, c = row - i * dx, col - i * dy
                if 0 <= r < self.board_size and 0 <= c < self.board_size and self.board[r][c] == self.current_player:
                    count += 1
                else:
                    break

            if count >= self.win_condition:
                return True
        return False

    def show_winner(self, winner):
        """Hiển thị thông báo khi có người thắng"""
        winner_text = f"Người chơi {winner} thắng!"
        self.canvas.create_text(self.board_size * 20, self.board_size * 20, text=winner_text, font=("Arial", 24), fill="blue")
        self.root.after(2000, self.reset_game)  # Quay lại màn hình chọn bàn cờ sau 2 giây

    def show_draw(self):
        """Hiển thị thông báo khi hòa"""
        draw_text = "Hòa!"
        self.canvas.create_text(self.board_size * 20, self.board_size * 20, text=draw_text, font=("Arial", 24), fill="blue")
        self.root.after(2000, self.reset_game)  # Quay lại màn hình chọn bàn cờ sau 2 giây

    def save_moves(self, winner=None):
        """Lưu dữ liệu vào dataset tương ứng với kích thước bàn cờ"""
        file_path = "caro_20x20.csv" if self.board_size == 20 else "caro_5x5.csv"

        df = pd.DataFrame(self.moves, columns=["MoveNumber", "Player", "Row", "Col"])
        df["Winner"] = winner if winner else 0  # Lưu thông tin người thắng (0 nếu hòa hoặc chưa có kết quả)
        df.to_csv(file_path, mode="a", header=False, index=False)

        print(f"Dữ liệu đã được lưu vào {file_path}")

    def reset_game(self):
        """Quay lại màn hình chọn bàn cờ"""
        for widget in self.root.winfo_children():
            widget.destroy()

        # Quay lại màn hình chọn bàn cờ
        self.__init__(self.root)

    def exit_game(self):
        """Thoát game"""
        self.root.destroy()  # Đóng hoàn toàn ứng dụng

# Chạy game
root = tk.Tk()
game = CaroGame(root)
root.mainloop()


<h1>Tạo data các quần cờ<h1>

In [3]:
import cv2
import numpy as np
import random
import os

# Hàm xoay ảnh
def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))
    return rotated_image

# Hàm thay đổi độ sáng
def adjust_brightness(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255)
    bright_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bright_image

# Hàm thay đổi độ tương phản
def adjust_contrast(image, factor):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    lab[:, :, 0] = np.clip(lab[:, :, 0] * factor, 0, 255)
    contrast_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_image

# Hàm thêm nhiễu
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 25
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy_image = np.uint8(np.clip(image + gauss, 0, 255))
    return noisy_image

# Hàm phản chiếu ảnh (flip)
def flip_image(image):
    flip_code = random.choice([-1, 0, 1])
    flipped_image = cv2.flip(image, flip_code)
    return flipped_image

# Tạo quân cờ X
def create_X_image(size=100):
    img = np.ones((size, size, 3), dtype=np.uint8) * 255
    cv2.line(img, (10, 10), (size-10, size-10), (0, 0, 0), 2)
    cv2.line(img, (size-10, 10), (10, size-10), (0, 0, 0), 2)
    return img

# Tạo quân cờ O
def create_O_image(size=100):
    img = np.ones((size, size, 3), dtype=np.uint8) * 255
    cv2.circle(img, (size//2, size//2), size//3, (0, 0, 0), 2)
    return img

# Tạo ô trống
def create_empty_image(size=100):
    img = np.ones((size, size, 3), dtype=np.uint8) * 255
    return img

# Tạo bộ dữ liệu tăng cường cho các quân cờ X, O và ô trống
def create_augmented_data(image, label, num_variations=10):
    augmented_images = []
    for _ in range(num_variations):
        # Xoay ảnh
        angle = random.randint(0, 360)
        rotated = rotate_image(image, angle)
        augmented_images.append((rotated, label))

        # Thay đổi độ sáng
        brightness_factor = random.uniform(0.5, 1.5)
        bright = adjust_brightness(image, brightness_factor)
        augmented_images.append((bright, label))

        # Thay đổi độ tương phản
        contrast_factor = random.uniform(0.5, 1.5)
        contrast = adjust_contrast(image, contrast_factor)
        augmented_images.append((contrast, label))

        # Thêm nhiễu
        noisy = add_noise(image)
        augmented_images.append((noisy, label))

        # Phản chiếu ảnh
        flipped = flip_image(image)
        augmented_images.append((flipped, label))

    return augmented_images

# Tạo thư mục để lưu ảnh
def create_folders():
    folders = ["X_images", "O_images", "empty_images"]
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)

# Lưu ảnh vào các thư mục tương ứng
def save_images(augmented_data, label):
    for i, (augmented_img, _) in enumerate(augmented_data):
        file_name = f"{label}_{i}.png"
        path = os.path.join(f"{label}_images", file_name)
        cv2.imwrite(path, augmented_img)

# Tạo và lưu bộ dữ liệu
def create_and_save_dataset():
    create_folders()

    # Tạo bộ dữ liệu cho quân cờ X, O và ô trống
    X_image = create_X_image()
    O_image = create_O_image()
    empty_image = create_empty_image()

    # Tạo bộ dữ liệu tăng cường cho từng loại
    X_augmented = create_augmented_data(X_image, 'X')
    O_augmented = create_augmented_data(O_image, 'O')
    empty_augmented = create_augmented_data(empty_image, 'empty')

    # Lưu các ảnh vào thư mục
    save_images(X_augmented, 'X')
    save_images(O_augmented, 'O')
    save_images(empty_augmented, 'empty')

    print("Bộ dữ liệu đã được tạo và lưu thành công!")

# Tạo và lưu bộ dữ liệu
create_and_save_dataset()


Bộ dữ liệu đã được tạo và lưu thành công!


<h1>Tạo data các bàn cờ<h1>

In [2]:
import cv2
import numpy as np
import random
import os

# Hàm xoay ảnh
# def rotate_image(image, angle):
#     rows, cols, _ = image.shape
#     M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
#     rotated_image = cv2.warpAffine(image, M, (cols, rows))
#     return rotated_image

import cv2
import numpy as np

# Hàm xoay ảnh không mất dữ liệu
def rotate_image(image, angle):
    rows, cols, _ = image.shape
    
    # Tính toán điểm giữa của ảnh
    center = (cols / 2, rows / 2)
    
    # Lấy ma trận xoay
    M = cv2.getRotationMatrix2D(center, angle, 1)
    
    # Tính toán kích thước ảnh mới sao cho chứa toàn bộ hình ảnh sau khi xoay
    abs_cos = abs(M[0, 0])
    abs_sin = abs(M[0, 1])
    
    # Tính kích thước mới của ảnh
    new_cols = int(rows * abs_sin + cols * abs_cos)
    new_rows = int(rows * abs_cos + cols * abs_sin)
    
    # Cập nhật ma trận xoay để chuyển về vị trí mới
    M[0, 2] += (new_cols / 2) - center[0]
    M[1, 2] += (new_rows / 2) - center[1]
    
    # Áp dụng xoay với kích thước mới
    rotated_image = cv2.warpAffine(image, M, (new_cols, new_rows))
    
    return rotated_image


# Hàm thay đổi độ sáng
def adjust_brightness(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255)
    bright_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bright_image

# Hàm thay đổi độ tương phản
def adjust_contrast(image, factor):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    lab[:, :, 0] = np.clip(lab[:, :, 0] * factor, 0, 255)
    contrast_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_image

# Hàm thêm nhiễu
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 25
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy_image = np.uint8(np.clip(image + gauss, 0, 255))
    return noisy_image

# Hàm phản chiếu ảnh (flip)
def flip_image(image):
    flip_code = random.choice([-1, 0, 1])  # -1: Lật cả chiều ngang và dọc, 0: Lật theo chiều dọc, 1: Lật theo chiều ngang
    flipped_image = cv2.flip(image, flip_code)
    return flipped_image

# Hàm tạo bàn cờ với kích thước n x n
def create_chessboard(n):
    # Kích thước mỗi ô trong bàn cờ
    size = 50
    
    # Kích thước của bàn cờ
    board_size = n * size
    
    # Tạo bàn cờ với nền trắng
    board = np.ones((board_size, board_size, 3), dtype=np.uint8) * 255

    # Vẽ các ô với đường kẻ đen
    for i in range(n):
        for j in range(n):
            # Vẽ ô với màu trắng (nền)
            cv2.rectangle(board, (i*size, j*size), ((i+1)*size, (j+1)*size), (255, 255, 255), -1)
            # Vẽ đường kẻ đen giữa các ô
            cv2.line(board, (i*size, 0), (i*size, board_size), (0, 0, 0), 2)  # Dọc
            cv2.line(board, (0, j*size), (board_size, j*size), (0, 0, 0), 2)  # Ngang

    # Vẽ đường kẻ ngoài cùng của bàn cờ
    cv2.line(board, (n*size-1, 0), (n*size-1, board_size), (0, 0, 0), 2)
    cv2.line(board, (0, n*size-1), (board_size, n*size-1), (0, 0, 0), 2)

    return board

# Hàm lưu bàn cờ và áp dụng các phép tăng cường
def save_chessboard_with_augmentation(num_variations=10):
    # Tạo các biến thể của bàn cờ với các phép tăng cường
    augmented_images = []
    
    # Tạo danh sách các góc xoay, độ sáng và độ tương phản có thể
    angles = [i * 15 for i in range(25)]  # Góc từ 0 đến 360, mỗi lần 15 độ
    brightness_factors = [0.2 + 0.2 * i for i in range(9)]  # Độ sáng từ 0.3 đến 2.0
    contrast_factors = [0.2 + 0.2 * i for i in range(9)]  # Độ tương phản từ 0.3 đến 2.0
    
    for n in range(3, 21):  # Tạo các bàn cờ từ 3x3 đến 20x20
        board = create_chessboard(n)
        
        # Trộn ngẫu nhiên các danh sách
        random.shuffle(angles)

        for _ in range(num_variations):
            # Chọn ngẫu nhiên từ danh sách đã trộn
            angle = random.choice(angles)
            rotated = rotate_image(board, angle)
            augmented_images.append((rotated, f"{n}x{n}"))

            brightness_factor = random.choice(brightness_factors)
            bright = adjust_brightness(board, brightness_factor)
            augmented_images.append((bright, f"{n}x{n}"))

            contrast_factor = random.choice(contrast_factors)
            contrast = adjust_contrast(board, contrast_factor)
            augmented_images.append((contrast, f"{n}x{n}"))

            # Thêm nhiễu
            noisy = add_noise(board)
            augmented_images.append((noisy, f"{n}x{n}"))

            # Phản chiếu ảnh
            flipped = flip_image(board)
            augmented_images.append((flipped, f"{n}x{n}"))

    return augmented_images

# Hàm kiểm tra và tạo tên thư mục duy nhất
def get_unique_folder_name(folder_name):
    counter = 1
    new_folder_name = folder_name
    while os.path.exists(new_folder_name):
        new_folder_name = f"{folder_name}({counter})"
        counter += 1
    return new_folder_name

# Hàm tạo thư mục để lưu ảnh
def create_folders():
    folder_name = "board_images"
    unique_folder_name = get_unique_folder_name(folder_name)
    os.makedirs(unique_folder_name)
    return unique_folder_name

# Lưu ảnh vào các thư mục tương ứng
def save_images(augmented_data, folder_name):
    for i, (augmented_img, label) in enumerate(augmented_data):
        file_name = f"{label}_{i}.png"
        path = os.path.join(folder_name, file_name)
        cv2.imwrite(path, augmented_img)

# Tạo và lưu bộ dữ liệu
def create_and_save_dataset():
    # Tạo thư mục duy nhất để lưu ảnh
    folder_name = create_folders()

    # Tạo bộ dữ liệu tăng cường cho từng loại
    board_augmented = save_chessboard_with_augmentation()

    # Lưu các ảnh vào thư mục
    save_images(board_augmented, folder_name)
    print(f"Bộ dữ liệu đã được tạo và lưu thành công tại thư mục {folder_name}!")

# Tạo và lưu bộ dữ liệu
create_and_save_dataset()


Bộ dữ liệu đã được tạo và lưu thành công tại thư mục board_images!
